##### Note: AIMM DB is frozen on an older version of Tiled for the moment (A postdoc left). It will be updated in May, but for now we need to use an older version of the Tiled client to work with it.

In [ ]:
import tiled
tiled.__version__ == "0.1.0a74"

## Connect to the AIMMdb server

In [ ]:
from tiled.client import from_uri

In [ ]:
c = from_uri("https://aimm.lbl.gov/api")
c

In [ ]:
c['dataset']

In [ ]:
c.login()

In [ ]:
list(c['dataset'])

## Find data and metadata

In [ ]:
#Total of samples in NMC
c['dataset']['nmc']['uid']

In [ ]:
c['dataset']['nmc']['element']['Ni']['edge']['K']['uid']['WwFMvg92n23'].read()

In [ ]:
c['dataset']['nmc']['element']['Ni']['edge']['K']['uid']['WwFMvg92n23'].metadata

In [ ]:
# Access collection of data to itereate from. You can pick from these options: .items(), .keys(), .values()
c['dataset']['nmc']['uid'].values().first()

In [ ]:
for key in c['dataset']['nmc']['element']['O']['edge']['K']['uid'].keys():
    print(key)

In [ ]:
c['dataset']['nmc']['uid'].distinct("facility.name", counts=True)

In [ ]:
from tiled.queries import Key

c['dataset']['nmc']['uid'].search(Key("element.symbol") == "Co").search(Key("facility.name") == "NSLS-II")

In [ ]:
element_list = ["Co", "Mn", "Ni"]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
als_dfs = {}
als_metadata = {}
for element in  element_list:
    als_data = c['dataset']['nmc']['uid'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "ALS").values().first()
    als_dfs[element] = als_data.read()
    als_metadata[element] = als_data.metadata

counter = 0
fig_als, ax_als = plt.subplots(1,3, figsize=(15, 4))
for key, df in als_dfs.items():
    if key == "Mn":
        ax_als[counter].plot(als_dfs[key]["energy"], als_dfs[key]["normfluor"], label=als_metadata[key]["element"]["symbol"])
        ax_als[counter].set(xlabel='energy', ylabel='normfluor')
    else:
        ax_als[counter].plot(als_dfs[key]["energy"], als_dfs[key]["mu_tfy"], label=als_metadata[key]["element"]["symbol"])
        ax_als[counter].set(xlabel='energy', ylabel='mu_tfy')
    ax_als[counter].legend()
    counter += 1
plt.show()

In [ ]:
aps_dfs = {}
aps_metadata = {}
for element in  element_list:
    aps_data = c['dataset']['nmc']['uid'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "APS").values().first()
    aps_dfs[element] = aps_data.read()
    aps_metadata[element] = aps_data.metadata

counter = 0
fig_aps, ax_aps = plt.subplots(1,3, figsize=(15, 4))
for key, df in aps_dfs.items():
    ax_aps[counter].plot(aps_dfs[key]["energy"], aps_dfs[key]["mutrans"], label=aps_metadata[key]["element"]["symbol"])
    ax_aps[counter].set(xlabel='energy', ylabel='mutrans')
    ax_aps[counter].legend()
    counter += 1
plt.show()

In [ ]:
nslsii_dfs = {}
nslsii_metadata = {}
for element in  element_list:
    nslsii_data = c['dataset']['nmc']['uid'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "NSLS-II").values().first()
    nslsii_dfs[element] = nslsii_data.read()
    nslsii_metadata[element] = nslsii_data.metadata
    
counter = 0
fig_nslsii, ax_nslsii = plt.subplots(1,3, figsize=(15, 4))
for key, df in nslsii_dfs.items():
    ax_nslsii[counter].plot(nslsii_dfs[key]["energy"], nslsii_dfs[key]["mutrans"], label=nslsii_metadata[key]["element"]["symbol"])
    ax_nslsii[counter].set(xlabel='energy', ylabel='mutrans')
    ax_nslsii[counter].legend()
    counter += 1
plt.show()

In [ ]:
fig_element, ax_element = plt.subplots(1,3, figsize=(15, 4))

counter = 0
for key in nslsii_dfs.keys():
    if key == "Mn":
        ax_element[counter].plot(als_dfs[key]["energy"], als_dfs[key]["normfluor"], label="ALS")
    else:
        ax_element[counter].plot(als_dfs[key]["energy"], als_dfs[key]["mu_tfy"], label="ALS")
    
    ax_element[counter].plot(aps_dfs[key]["energy"], aps_dfs[key]["mutrans"], label="APS")
    ax_element[counter].plot(nslsii_dfs[key]["energy"], nslsii_dfs[key]["mutrans"], label="NSLS-II")
    ax_element[counter].set(title=key)
    
    ax_element[counter].legend()
    counter += 1
plt.show()

## Share data with collaboration

In [ ]:
import pandas as pd

In [ ]:
data = {"node_id": [1, 2, 3],
       "probability": [0.2, 0.7, 0.1]}
df = pd.DataFrame(data)

metadata = {"dataset": "demo",
            "element": {"edge": "K", "symbol": "Ni"},
            "facility": "NSLS-II",
           }

In [ ]:
df_node = c["uid"].write_dataframe(df, metadata)
df_node

In [ ]:
node_id = df_node.item["id"]
c["uid"][node_id]

In [ ]:
list(c['dataset'])

In [ ]:
c["uid"][node_id].export("demo.csv")

In [ ]:
del c["uid"][node_id]